In [1]:
import cx_Oracle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
plt.rc('font', family = 'Malgun Gothic') # 한글 입력
plt.rcParams['axes.unicode_minus'] = False # 글꼴 변경시 마이너스 표기 가능으로 처리

## 카테고리별 ( sum,count).csv

In [9]:
def get_category(learning,way):
    #계절성 제거
    def new_seasonal_scaled(df):
        df1 = df.copy()
        if way=='sum':
            df1.loc[df1['기'] == '1기', '총구매액'] = np.round(df1.loc[df1['기'] == '1기', '총구매액'] / 0.9476399112)
            df1.loc[df1['기'] == '2기', '총구매액'] = np.round(df1.loc[df1['기'] == '2기', '총구매액'] / 0.9785290501)    
            df1.loc[df1['기'] == '3기', '총구매액'] = np.round(df1.loc[df1['기'] == '3기', '총구매액'] / 0.9072731695)    
            df1.loc[df1['기'] == '4기', '총구매액'] = np.round(df1.loc[df1['기'] == '4기', '총구매액'] / 1.166557869)    
            df1.loc[df1['기'] == '5기', '총구매액'] = np.round(df1.loc[df1['기'] == '5기', '총구매액'] / 0.9748289537)    
            df1.loc[df1['기'] == '6기', '총구매액'] = np.round(df1.loc[df1['기'] == '6기', '총구매액'] / 0.9784250528)    
            df1.loc[df1['기'] == '7기', '총구매액'] = np.round(df1.loc[df1['기'] == '7기', '총구매액'] / 0.912781587)    
            df1.loc[df1['기'] == '8기', '총구매액'] = np.round(df1.loc[df1['기'] == '8기', '총구매액'] / 1.133964406)
        else:
            print(way)
            df1.loc[df1['기'] == '1기', '총구매액'] = np.round(df1.loc[df1['기'] == '1기', '총구매액'] / 0.9278494042)
            df1.loc[df1['기'] == '2기', '총구매액'] = np.round(df1.loc[df1['기'] == '2기', '총구매액'] / 1.006872934)    
            df1.loc[df1['기'] == '3기', '총구매액'] = np.round(df1.loc[df1['기'] == '3기', '총구매액'] / 1.02764673)    
            df1.loc[df1['기'] == '4기', '총구매액'] = np.round(df1.loc[df1['기'] == '4기', '총구매액'] / 1.037630932)    
            df1.loc[df1['기'] == '5기', '총구매액'] = np.round(df1.loc[df1['기'] == '5기', '총구매액'] / 0.9833832331)    
            df1.loc[df1['기'] == '6기', '총구매액'] = np.round(df1.loc[df1['기'] == '6기', '총구매액'] / 1.047412508)    
            df1.loc[df1['기'] == '7기', '총구매액'] = np.round(df1.loc[df1['기'] == '7기', '총구매액'] / 1.031315797)    
            df1.loc[df1['기'] == '8기', '총구매액'] = np.round(df1.loc[df1['기'] == '8기', '총구매액'] / 0.937888462)
        df1['총구매액'] = df1['총구매액'].astype('int64')
        return df1
    
    #df 불러오기
    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    sql = f'''SELECT a.고객번호, a.분류, b.고객구분, a.기, {way}(a.구매금액) "총구매액"
    FROM LCL2 a
    JOIN 고정고객 b ON a.고객번호 = b.고객번호
    GROUP BY a.고객번호, a.분류, b.고객구분, a.기
    ORDER BY a.고객번호, a.분류, b.고객구분, a.기'''
    cursor.execute(sql)
    x = cursor.fetchall()
    columns = ['고객번호','분류', '고객구분', '기', '총구매액']
    df_all = pd.DataFrame(x, columns = columns)
    cursor.close()
    con.commit()
    con.close()

    df_all_scaled = new_seasonal_scaled(df_all)
    df_all_scaled.head()

    cusno = df_all_scaled['고객번호'].unique().tolist()
    cat = df_all_scaled['분류'].unique().tolist()
    season = df_all_scaled['기'].unique().tolist()

    cusno_re = []
    cat_re = []
    season_re = []

    for i in np.arange(len(cat)*len(season)):
        for j in cusno:
            cusno_re.append(j)

    for i in np.arange(int(len(cusno_re) / len(cat))):
        for j in cat:
            cat_re.append(j)

    for i in np.arange(int(len(cusno_re) / len(season))):
        for j in season:
            season_re.append(j)

    cusno_re.sort()

    df_all_re = pd.DataFrame()
    df_all_re['고객번호'] = cusno_re
    df_all_re['분류'] = cat_re
    df_all_re = df_all_re.sort_values(['고객번호', '분류'])
    df_all_re['기'] = season_re
    df_all_re['총구매액'] = 0

    df_all_scaled_p = pd.merge(df_all_re, df_all_scaled, how = 'outer', on = ['고객번호', '분류', '기'])
    
    if learning=='train':
        learning='train'
    else:
        learning='test'
    
    if way =='sum':
        way='구매'
    else:
        way='방문'
    #데이터전처리
    df_all_scaled_p.fillna(0,inplace=True)
    df_all_scaled_p.drop(['총구매액_x','고객구분'],axis=1,inplace=True)
    df_all_scaled_p.columns=['고객번호','분류','기','총구매액']
    df=df_all_scaled_p
    df['총구매액']=df['총구매액'].astype('int64')
    df.총구매액=np.log1p(df.총구매액)
    
    #복사본 생성, 계절성 제거
    df_quarter=df.copy()
    # 결과물을 위한 데이터 프레임
    custom=pd.Series(df_quarter.고객번호.unique())

    df_result=pd.DataFrame(data=custom,columns=['고객번호']) 
    df_final=pd.DataFrame(data=custom,columns=['고객번호']) 

    labels=['가공식품','신선식품','일상용품','의약품/의료기기','교육/문화용품','외식','기타',
           '디지털/가전','가구/인테리어','의류','전문스포츠/레저','패션잡화','유아동',
           '명품']

    for label in labels:
        for i in df_quarter.기.unique():
            df_temp=df_quarter[(df_quarter.기==i) & (df_quarter.분류==label)]
            df_temp=df_temp.reset_index()
            df_result[f"{label}_{i}"] = pd.cut(df_temp['총구매액'], bins=10, labels=[1,2,3,4,5,6,7,8,9,10]).astype("int64")
            
        if learning=='train':
            df_result[f'{label}_증감'] = df_result[f"{label}_6기"]-df_result[f"{label}_1기"]
            df_result[f'{label}_변동']=abs(df_result[f'{label}_2기']-df_result[f'{label}_1기'])+\
            abs(df_result[f'{label}_3기']-df_result[f'{label}_2기'])+\
            abs(df_result[f'{label}_4기']-df_result[f'{label}_3기'])+\
            abs(df_result[f'{label}_5기']-df_result[f'{label}_4기'])+\
            abs(df_result[f'{label}_6기']-df_result[f'{label}_5기'])
        else:
            df_result[f'{label}_증감'] = df_result[f"{label}_7기"]-df_result[f"{label}_1기"]
            df_result[f'{label}_변동']=abs(df_result[f'{label}_2기']-df_result[f'{label}_1기'])+\
            abs(df_result[f'{label}_3기']-df_result[f'{label}_2기'])+\
            abs(df_result[f'{label}_4기']-df_result[f'{label}_3기'])+\
            abs(df_result[f'{label}_5기']-df_result[f'{label}_4기'])+\
            abs(df_result[f'{label}_6기']-df_result[f'{label}_5기'])+\
            abs(df_result[f'{label}_7기']-df_result[f'{label}_6기'])
        df_final[f'{label}_{way}지수']=df_result[f'{label}_증감']*0.8+df_result[f'{label}_변동']*0.2
    return df_final

In [10]:
get_category('train','sum').to_csv(f'카테고리_구매지수(1-6).csv',encoding='cp949',index=False)
get_category('test','sum').to_csv(f'카테고리_구매지수(1-7).csv',encoding='cp949',index=False)
get_category('train','count').to_csv(f'카테고리_방문지수(1-6).csv',encoding='cp949',index=False)
get_category('test','count').to_csv(f'카테고리_방문지수(1-7).csv',encoding='cp949',index=False)

count
count


## 제휴사별 ( sum,count),csv

In [11]:
def get_affiliate(learning,way):
    #계절성 제거
    def new_seasonal_scaled(df):
        df1 = df.copy()
        if way=='sum':
            df1.loc[df1['기'] == '1기', '총구매액'] = np.round(df1.loc[df1['기'] == '1기', '총구매액'] / 0.9476399112)
            df1.loc[df1['기'] == '2기', '총구매액'] = np.round(df1.loc[df1['기'] == '2기', '총구매액'] / 0.9785290501)    
            df1.loc[df1['기'] == '3기', '총구매액'] = np.round(df1.loc[df1['기'] == '3기', '총구매액'] / 0.9072731695)    
            df1.loc[df1['기'] == '4기', '총구매액'] = np.round(df1.loc[df1['기'] == '4기', '총구매액'] / 1.166557869)    
            df1.loc[df1['기'] == '5기', '총구매액'] = np.round(df1.loc[df1['기'] == '5기', '총구매액'] / 0.9748289537)    
            df1.loc[df1['기'] == '6기', '총구매액'] = np.round(df1.loc[df1['기'] == '6기', '총구매액'] / 0.9784250528)    
            df1.loc[df1['기'] == '7기', '총구매액'] = np.round(df1.loc[df1['기'] == '7기', '총구매액'] / 0.912781587)    
            df1.loc[df1['기'] == '8기', '총구매액'] = np.round(df1.loc[df1['기'] == '8기', '총구매액'] / 1.133964406)
        else:
            print(way)
            df1.loc[df1['기'] == '1기', '총구매액'] = np.round(df1.loc[df1['기'] == '1기', '총구매액'] / 0.9278494042)
            df1.loc[df1['기'] == '2기', '총구매액'] = np.round(df1.loc[df1['기'] == '2기', '총구매액'] / 1.006872934)    
            df1.loc[df1['기'] == '3기', '총구매액'] = np.round(df1.loc[df1['기'] == '3기', '총구매액'] / 1.02764673)    
            df1.loc[df1['기'] == '4기', '총구매액'] = np.round(df1.loc[df1['기'] == '4기', '총구매액'] / 1.037630932)    
            df1.loc[df1['기'] == '5기', '총구매액'] = np.round(df1.loc[df1['기'] == '5기', '총구매액'] / 0.9833832331)    
            df1.loc[df1['기'] == '6기', '총구매액'] = np.round(df1.loc[df1['기'] == '6기', '총구매액'] / 1.047412508)    
            df1.loc[df1['기'] == '7기', '총구매액'] = np.round(df1.loc[df1['기'] == '7기', '총구매액'] / 1.031315797)    
            df1.loc[df1['기'] == '8기', '총구매액'] = np.round(df1.loc[df1['기'] == '8기', '총구매액'] / 0.937888462)
        df1['총구매액'] = df1['총구매액'].astype('int64')
        return df1
    
    #df 불러오기
    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    sql = f'''SELECT a.고객번호, 제휴사, b.고객구분, a.기, {way}(a.구매금액) "총구매액"
    FROM LCL2 a
    JOIN 고정고객 b ON a.고객번호 = b.고객번호
    GROUP BY a.고객번호, 제휴사, b.고객구분, a.기
    ORDER BY a.고객번호, 제휴사, b.고객구분, a.기
    '''
    cursor.execute(sql)
    x = cursor.fetchall()
    columns = ['고객번호','제휴사', '고객구분', '기', '총구매액']
    df_all = pd.DataFrame(x, columns = columns)
    cursor.close()
    con.commit()
    con.close()

    display(df_all.head())

    df_all_scaled = new_seasonal_scaled(df_all)
    df_all_scaled.head()

    cusno = df_all_scaled['고객번호'].unique().tolist()
    cat = df_all_scaled['제휴사'].unique().tolist()
    season = df_all_scaled['기'].unique().tolist()

    cusno_re = []
    cat_re = []
    season_re = []

    for i in np.arange(len(cat)*len(season)):
        for j in cusno:
            cusno_re.append(j)

    for i in np.arange(int(len(cusno_re) / len(cat))):
        for j in cat:
            cat_re.append(j)

    for i in np.arange(int(len(cusno_re) / len(season))):
        for j in season:
            season_re.append(j)

    cusno_re.sort()

    df_all_re = pd.DataFrame()
    df_all_re['고객번호'] = cusno_re
    df_all_re['제휴사'] = cat_re
    df_all_re = df_all_re.sort_values(['고객번호', '제휴사'])
    df_all_re['기'] = season_re
    df_all_re['총구매액'] = 0

    df_all_scaled_p = pd.merge(df_all_re, df_all_scaled, how = 'outer', on = ['고객번호', '제휴사', '기'])
    df_all_scaled_p
    
    #데이터전처리
    df_all_scaled_p.fillna(0,inplace=True)
    df_all_scaled_p.drop(['총구매액_x','고객구분'],axis=1,inplace=True)
    df_all_scaled_p.columns=['고객번호','제휴사','기','총구매액']
    df=df_all_scaled_p
    df['총구매액']=df['총구매액'].astype('int64')
    df.총구매액=np.log1p(df.총구매액)
    
    #복사본 생성, 계절성 제거
    df_quarter=df.copy()
    # 결과물을 위한 데이터 프레임
    custom=pd.Series(df_quarter.고객번호.unique())

    df_result=pd.DataFrame(data=custom,columns=['고객번호']) 
    df_final=pd.DataFrame(data=custom,columns=['고객번호']) 

    labels=['A','B','C','D']

    for label in labels:
        for i in df_quarter.기.unique():
            df_temp=df_quarter[(df_quarter.기==i) & (df_quarter.제휴사==label)]
            df_temp=df_temp.reset_index()
            df_result[f"{label}_{i}"] = pd.cut(df_temp['총구매액'], bins=10, labels=[1,2,3,4,5,6,7,8,9,10]).astype("int64")
            
        if learning=='train':
            df_result[f'{label}_증감'] = df_result[f"{label}_6기"]-df_result[f"{label}_1기"]
            df_result[f'{label}_변동']=abs(df_result[f'{label}_2기']-df_result[f'{label}_1기'])+\
            abs(df_result[f'{label}_3기']-df_result[f'{label}_2기'])+\
            abs(df_result[f'{label}_4기']-df_result[f'{label}_3기'])+\
            abs(df_result[f'{label}_5기']-df_result[f'{label}_4기'])+\
            abs(df_result[f'{label}_6기']-df_result[f'{label}_5기'])
        else:
            df_result[f'{label}_증감'] = df_result[f"{label}_7기"]-df_result[f"{label}_1기"]
            df_result[f'{label}_변동']=abs(df_result[f'{label}_2기']-df_result[f'{label}_1기'])+\
            abs(df_result[f'{label}_3기']-df_result[f'{label}_2기'])+\
            abs(df_result[f'{label}_4기']-df_result[f'{label}_3기'])+\
            abs(df_result[f'{label}_5기']-df_result[f'{label}_4기'])+\
            abs(df_result[f'{label}_6기']-df_result[f'{label}_5기'])+\
            abs(df_result[f'{label}_7기']-df_result[f'{label}_6기'])
        df_final[f'{label}_{way}지수']=df_result[f'{label}_증감']*0.85+df_result[f'{label}_변동']*0.15
    return df_final

In [12]:
get_affiliate('train','sum').to_csv('제휴사_구매지수(1-6).csv',encoding='cp949',index=False)
get_affiliate('test','sum').to_csv('제휴사_구매지수(1-7).csv',encoding='cp949',index=False)
get_affiliate('train','count').to_csv('제휴사_방문지수(1-6).csv',encoding='cp949',index=False)
get_affiliate('test','count').to_csv('제휴사_방문지수(1-7).csv',encoding='cp949',index=False)

,고객번호,제휴사,고객구분,기,총구매액
0,00001,A,감소,1기,9119790
1,00001,A,감소,2기,10583420
2,00001,A,감소,3기,10722160
3,00001,A,감소,4기,22618140
4,00001,A,감소,5기,9007070


,고객번호,제휴사,고객구분,기,총구매액
0,00001,A,감소,1기,9119790
1,00001,A,감소,2기,10583420
2,00001,A,감소,3기,10722160
3,00001,A,감소,4기,22618140
4,00001,A,감소,5기,9007070


,고객번호,제휴사,고객구분,기,총구매액
0,00001,A,감소,1기,120
1,00001,A,감소,2기,110
2,00001,A,감소,3기,129
3,00001,A,감소,4기,145
4,00001,A,감소,5기,138


count


,고객번호,제휴사,고객구분,기,총구매액
0,00001,A,감소,1기,120
1,00001,A,감소,2기,110
2,00001,A,감소,3기,129
3,00001,A,감소,4기,145
4,00001,A,감소,5기,138


count


## 전체 분기 매출(sum,count).csv

In [13]:
def get_month(learning,way):
    #계절성 제거
    def new_seasonal_scaled(df):
        df1 = df.copy()
        if way=='sum':
            df1.loc[df1['기'] == '1기', '총구매액'] = np.round(df1.loc[df1['기'] == '1기', '총구매액'] / 0.9476399112)
            df1.loc[df1['기'] == '2기', '총구매액'] = np.round(df1.loc[df1['기'] == '2기', '총구매액'] / 0.9785290501)    
            df1.loc[df1['기'] == '3기', '총구매액'] = np.round(df1.loc[df1['기'] == '3기', '총구매액'] / 0.9072731695)    
            df1.loc[df1['기'] == '4기', '총구매액'] = np.round(df1.loc[df1['기'] == '4기', '총구매액'] / 1.166557869)    
            df1.loc[df1['기'] == '5기', '총구매액'] = np.round(df1.loc[df1['기'] == '5기', '총구매액'] / 0.9748289537)    
            df1.loc[df1['기'] == '6기', '총구매액'] = np.round(df1.loc[df1['기'] == '6기', '총구매액'] / 0.9784250528)    
            df1.loc[df1['기'] == '7기', '총구매액'] = np.round(df1.loc[df1['기'] == '7기', '총구매액'] / 0.912781587)    
            df1.loc[df1['기'] == '8기', '총구매액'] = np.round(df1.loc[df1['기'] == '8기', '총구매액'] / 1.133964406)
        else:
            print(way)
            df1.loc[df1['기'] == '1기', '총구매액'] = np.round(df1.loc[df1['기'] == '1기', '총구매액'] / 0.9278494042)
            df1.loc[df1['기'] == '2기', '총구매액'] = np.round(df1.loc[df1['기'] == '2기', '총구매액'] / 1.006872934)    
            df1.loc[df1['기'] == '3기', '총구매액'] = np.round(df1.loc[df1['기'] == '3기', '총구매액'] / 1.02764673)    
            df1.loc[df1['기'] == '4기', '총구매액'] = np.round(df1.loc[df1['기'] == '4기', '총구매액'] / 1.037630932)    
            df1.loc[df1['기'] == '5기', '총구매액'] = np.round(df1.loc[df1['기'] == '5기', '총구매액'] / 0.9833832331)    
            df1.loc[df1['기'] == '6기', '총구매액'] = np.round(df1.loc[df1['기'] == '6기', '총구매액'] / 1.047412508)    
            df1.loc[df1['기'] == '7기', '총구매액'] = np.round(df1.loc[df1['기'] == '7기', '총구매액'] / 1.031315797)    
            df1.loc[df1['기'] == '8기', '총구매액'] = np.round(df1.loc[df1['기'] == '8기', '총구매액'] / 0.937888462)
        df1['총구매액'] = df1['총구매액'].astype('int64')
        return df1
    
    #df 불러오기
    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    sql = f'''SELECT a.고객번호 , a.기, {way}(a.구매금액) "총구매액"
    FROM LCL2 a
    JOIN 고정고객 b ON a.고객번호 = b.고객번호
    GROUP BY a.고객번호, 제휴사, a.기
    ORDER BY a.고객번호, 제휴사, a.기
    '''
    cursor.execute(sql)
    x = cursor.fetchall()
    columns = ['고객번호', '기', '총구매액']
    df_all = pd.DataFrame(x, columns = columns)
    cursor.close()
    con.commit()
    con.close()

    display(df_all.head())

    df_all_scaled = new_seasonal_scaled(df_all)
    df_all_scaled.head()

    cusno = df_all_scaled['고객번호'].unique().tolist()
    season = df_all_scaled['기'].unique().tolist()

    cusno_re = []
    season_re = []

    for i in np.arange(int(len(cusno_re) / len(season))):
        for j in season:
            season_re.append(j)

    cusno_re.sort()

    df_all_re = pd.DataFrame()
    df_all_re['고객번호'] = cusno_re
    df_all_re = df_all_re.sort_values(['고객번호'])
    df_all_re['기'] = season_re
    df_all_re['총구매액'] = 0

    df_all_scaled_p = pd.merge(df_all_re, df_all_scaled, how = 'outer', on = ['고객번호', '기'])
    df_all_scaled_p
    
    #데이터전처리
    df_all_scaled_p.fillna(0,inplace=True)
    df_all_scaled_p.drop(['총구매액_x'],axis=1,inplace=True)
    df_all_scaled_p.columns=['고객번호','기','총구매액']
    df=df_all_scaled_p
    df['총구매액']=df['총구매액'].astype('int64')
    df.총구매액=np.log1p(df.총구매액)
    
    #복사본 생성, 계절성 제거
    df_quarter=df.copy()
    # 결과물을 위한 데이터 프레임
    custom=pd.Series(df_quarter.고객번호.unique())
    df_result=pd.DataFrame(data=custom,columns=['고객번호']) 
    df_final=pd.DataFrame(data=custom,columns=['고객번호']) 
    
    label='분기'
    for i in df_quarter.기.unique():
        df_temp=df_quarter[df_quarter.기==i]
        df_temp=df_temp.reset_index()
        df_result[f"{label}_{i}"] = pd.cut(df_temp['총구매액'], bins=10, labels=[1,2,3,4,5,6,7,8,9,10]).astype("int64")

    if learning=='train':
        df_result[f'{label}_증감'] = df_result[f"{label}_6기"]-df_result[f"{label}_1기"]
        df_result[f'{label}_변동']=abs(df_result[f'{label}_2기']-df_result[f'{label}_1기'])+\
        abs(df_result[f'{label}_3기']-df_result[f'{label}_2기'])+\
        abs(df_result[f'{label}_4기']-df_result[f'{label}_3기'])+\
        abs(df_result[f'{label}_5기']-df_result[f'{label}_4기'])+\
        abs(df_result[f'{label}_6기']-df_result[f'{label}_5기'])
    else:
        df_result[f'{label}_증감'] = df_result[f"{label}_7기"]-df_result[f"{label}_1기"]
        df_result[f'{label}_변동']=abs(df_result[f'{label}_2기']-df_result[f'{label}_1기'])+\
        abs(df_result[f'{label}_3기']-df_result[f'{label}_2기'])+\
        abs(df_result[f'{label}_4기']-df_result[f'{label}_3기'])+\
        abs(df_result[f'{label}_5기']-df_result[f'{label}_4기'])+\
        abs(df_result[f'{label}_6기']-df_result[f'{label}_5기'])+\
        abs(df_result[f'{label}_7기']-df_result[f'{label}_6기'])
    df_final[f'{label}_{way}지수']=df_result[f'{label}_증감']*0.85+df_result[f'{label}_변동']*0.15
    
    return df_final

In [14]:
get_month('train','sum').to_csv('분기_구매지수(1-6).csv',encoding='cp949',index=False)
get_month('test','sum').to_csv('분기_구매지수(1-7).csv',encoding='cp949',index=False)
get_month('train','count').to_csv('분기_방문지수(1-6).csv',encoding='cp949',index=False)
get_month('test','count').to_csv('분기_방문지수(1-7).csv',encoding='cp949',index=False)

,고객번호,기,총구매액
0,00001,1기,9119790
1,00001,2기,10583420
2,00001,3기,10722160
3,00001,4기,22618140
4,00001,5기,9007070


,고객번호,기,총구매액
0,00001,1기,9119790
1,00001,2기,10583420
2,00001,3기,10722160
3,00001,4기,22618140
4,00001,5기,9007070


,고객번호,기,총구매액
0,00001,1기,120
1,00001,2기,110
2,00001,3기,129
3,00001,4기,145
4,00001,5기,138


count


,고객번호,기,총구매액
0,00001,1기,120
1,00001,2기,110
2,00001,3기,129
3,00001,4기,145
4,00001,5기,138


count


## 주요 피쳐 세분화.csv

In [8]:
def get_category(learning,way,name):
    #계절성 제거
    def new_seasonal_scaled(df):
        df1 = df.copy()
        df1.loc[df1['기'] == '1기', '총구매액'] = np.round(df1.loc[df1['기'] == '1기', '총구매액'] / 0.9476399112)
        df1.loc[df1['기'] == '2기', '총구매액'] = np.round(df1.loc[df1['기'] == '2기', '총구매액'] / 0.9785290501)    
        df1.loc[df1['기'] == '3기', '총구매액'] = np.round(df1.loc[df1['기'] == '3기', '총구매액'] / 0.9072731695)    
        df1.loc[df1['기'] == '4기', '총구매액'] = np.round(df1.loc[df1['기'] == '4기', '총구매액'] / 1.166557869)    
        df1.loc[df1['기'] == '5기', '총구매액'] = np.round(df1.loc[df1['기'] == '5기', '총구매액'] / 0.9748289537)    
        df1.loc[df1['기'] == '6기', '총구매액'] = np.round(df1.loc[df1['기'] == '6기', '총구매액'] / 0.9784250528)    
        df1.loc[df1['기'] == '7기', '총구매액'] = np.round(df1.loc[df1['기'] == '7기', '총구매액'] / 0.912781587)    
        df1.loc[df1['기'] == '8기', '총구매액'] = np.round(df1.loc[df1['기'] == '8기', '총구매액'] / 1.133964406)
        df1['총구매액'] = df1['총구매액'].astype('int64')
        return df1
    
    #df 불러오기
    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    sql = f'''SELECT a.고객번호,a.분류등급,b.고객구분, a.기, {way}(a.구매금액) "총구매액"
FROM LCL2 a 
JOIN 고정고객 b ON a.고객번호 = b.고객번호
where 분류등급 = '{name}'
GROUP BY a.고객번호, a.분류등급, b.고객구분, a.기
ORDER BY a.고객번호, a.분류등급, b.고객구분, a.기'''
    cursor.execute(sql)
    x = cursor.fetchall()
    columns = ['고객번호','분류','고객구분', '기', '총구매액']
    df_all = pd.DataFrame(x, columns = columns)
    cursor.close()
    con.commit()
    con.close()

    df_all_scaled = new_seasonal_scaled(df_all)
    df_all_scaled.head()

    cusno = df_all_scaled['고객번호'].unique().tolist()
    cat = df_all_scaled['분류'].unique().tolist()
    season = df_all_scaled['기'].unique().tolist()

    cusno_re = []
    cat_re = []
    season_re = []

    for i in np.arange(len(cat)*len(season)):
        for j in cusno:
            cusno_re.append(j)

    for i in np.arange(int(len(cusno_re) / len(cat))):
        for j in cat:
            cat_re.append(j)

    for i in np.arange(int(len(cusno_re) / len(season))):
        for j in season:
            season_re.append(j)

    cusno_re.sort()

    df_all_re = pd.DataFrame()
    df_all_re['고객번호'] = cusno_re
    df_all_re['분류'] = cat_re
    df_all_re = df_all_re.sort_values(['고객번호', '분류'])
    df_all_re['기'] = season_re
    df_all_re['총구매액'] = 0

    df_all_scaled_p = pd.merge(df_all_re, df_all_scaled, how = 'outer', on = ['고객번호', '분류', '기'])
    
    if learning=='train':
        learning='train'
    else:
        learning='test'
    
    if way =='sum':
        way='구매'
    else:
        way='방문'
    #데이터전처리
    df_all_scaled_p.fillna(0,inplace=True)
    df_all_scaled_p.drop(['총구매액_x','고객구분'],axis=1,inplace=True)
    df_all_scaled_p.columns=['고객번호','분류','기','총구매액']
    df=df_all_scaled_p
    df['총구매액']=df['총구매액'].astype('int64')
    df.총구매액=np.log1p(df.총구매액)
    
    #복사본 생성, 계절성 제거
    df_quarter=df.copy()
    # 결과물을 위한 데이터 프레임
    custom=pd.Series(df_quarter.고객번호.unique())

    df_result=pd.DataFrame(data=custom,columns=['고객번호']) 
    df_final=pd.DataFrame(data=custom,columns=['고객번호']) 

    labels=[name]

    for label in labels:
        for i in df_quarter.기.unique():
            df_temp=df_quarter[(df_quarter.기==i) & (df_quarter.분류==label)]
            df_temp=df_temp.reset_index()
            df_result[f"{label}_{i}"] = pd.cut(df_temp['총구매액'], bins=10, labels=[1,2,3,4,5,6,7,8,9,10]).astype("int64")
            
        if learning=='train':
            df_result[f'{label}_증감'] = df_result[f"{label}_6기"]-df_result[f"{label}_1기"]
            df_result[f'{label}_변동']=abs(df_result[f'{label}_2기']-df_result[f'{label}_1기'])+\
            abs(df_result[f'{label}_3기']-df_result[f'{label}_2기'])+\
            abs(df_result[f'{label}_4기']-df_result[f'{label}_3기'])+\
            abs(df_result[f'{label}_5기']-df_result[f'{label}_4기'])+\
            abs(df_result[f'{label}_6기']-df_result[f'{label}_5기'])
        else:
            df_result[f'{label}_증감'] = df_result[f"{label}_7기"]-df_result[f"{label}_1기"]
            df_result[f'{label}_변동']=abs(df_result[f'{label}_2기']-df_result[f'{label}_1기'])+\
            abs(df_result[f'{label}_3기']-df_result[f'{label}_2기'])+\
            abs(df_result[f'{label}_4기']-df_result[f'{label}_3기'])+\
            abs(df_result[f'{label}_5기']-df_result[f'{label}_4기'])+\
            abs(df_result[f'{label}_6기']-df_result[f'{label}_5기'])+\
            abs(df_result[f'{label}_7기']-df_result[f'{label}_6기'])
        df_final[f'{label}_{way}지수']=df_result[f'{label}_증감']*0.8+df_result[f'{label}_변동']*0.2
    return df_final

In [18]:
get_category('train','sum','일상용품_상').to_csv('일상용품_상_구매지수(1-6).csv',encoding='cp949',index=False)
get_category('train','sum','일상용품_중').to_csv('일상용품_중_구매지수(1-6).csv',encoding='cp949',index=False)
get_category('train','sum','일상용품_하').to_csv('일상용품_하_구매지수(1-6).csv',encoding='cp949',index=False)

get_category('test','sum','일상용품_상').to_csv('일상용품_상_구매지수(1-7).csv',encoding='cp949',index=False)
get_category('test','sum','일상용품_중').to_csv('일상용품_중_구매지수(1-7).csv',encoding='cp949',index=False)
get_category('test','sum','일상용품_하').to_csv('일상용품_하_구매지수(1-7).csv',encoding='cp949',index=False)

In [14]:
get_category('train','count','일상용품_상').to_csv('일상용품_상_방문지수(1-6).csv',encoding='cp949',index=False)
get_category('train','count','일상용품_중').to_csv('일상용품_중_방문지수(1-6).csv',encoding='cp949',index=False)
get_category('train','count','일상용품_하').to_csv('일상용품_하_방문지수(1-6).csv',encoding='cp949',index=False)

get_category('test','count','일상용품_상').to_csv('일상용품_상_방문지수(1-7).csv',encoding='cp949',index=False)
get_category('test','count','일상용품_중').to_csv('일상용품_중_방문지수(1-7).csv',encoding='cp949',index=False)
get_category('test','count','일상용품_하').to_csv('일상용품_하_방문지수(1-7).csv',encoding='cp949',index=False)

In [19]:
get_category('train','count','패션잡화_상').to_csv('패션잡화_상_방문지수(1-6).csv',encoding='cp949',index=False)
get_category('train','count','패션잡화_중').to_csv('패션잡화_중_방문지수(1-6).csv',encoding='cp949',index=False)
get_category('train','count','패션잡화_하').to_csv('패션잡화_하_방문지수(1-6).csv',encoding='cp949',index=False)

get_category('test','count','패션잡화_상').to_csv('패션잡화_상_방문지수(1-7).csv',encoding='cp949',index=False)
get_category('test','count','패션잡화_중').to_csv('패션잡화_중_방문지수(1-7).csv',encoding='cp949',index=False)
get_category('test','count','패션잡화_하').to_csv('패션잡화_하_방문지수(1-7).csv',encoding='cp949',index=False)


In [15]:
get_category('train','sum','패션잡화_상').to_csv('패션잡화_상_구매지수(1-6).csv',encoding='cp949',index=False)
get_category('train','sum','패션잡화_중').to_csv('패션잡화_중_구매지수(1-6).csv',encoding='cp949',index=False)
get_category('train','sum','패션잡화_하').to_csv('패션잡화_하_구매지수(1-6).csv',encoding='cp949',index=False)

get_category('test','sum','패션잡화_상').to_csv('패션잡화_상_구매지수(1-7).csv',encoding='cp949',index=False)
get_category('test','sum','패션잡화_중').to_csv('패션잡화_중_구매지수(1-7).csv',encoding='cp949',index=False)
get_category('test','sum','패션잡화_하').to_csv('패션잡화_하_구매지수(1-7).csv',encoding='cp949',index=False)


In [20]:
get_category('train','count','의류_상').to_csv('의류_상_방문지수(1-6).csv',encoding='cp949',index=False)
get_category('train','count','의류_중').to_csv('의류_중_방문지수(1-6).csv',encoding='cp949',index=False)
get_category('train','count','의류_하').to_csv('의류_하_방문지수(1-6).csv',encoding='cp949',index=False)

get_category('test','count','의류_상').to_csv('의류_상_방문지수(1-7).csv',encoding='cp949',index=False)
get_category('test','count','의류_중').to_csv('의류_중_방문지수(1-7).csv',encoding='cp949',index=False)
get_category('test','count','의류_하').to_csv('의류_하_방문지수(1-7).csv',encoding='cp949',index=False)


In [16]:
get_category('train','sum','의류_상').to_csv('의류_상_구매지수(1-6).csv',encoding='cp949',index=False)
get_category('train','sum','의류_중').to_csv('의류_중_구매지수(1-6).csv',encoding='cp949',index=False)
get_category('train','sum','의류_하').to_csv('의류_하_구매지수(1-6).csv',encoding='cp949',index=False)

get_category('test','sum','의류_상').to_csv('의류_상_구매지수(1-7).csv',encoding='cp949',index=False)
get_category('test','sum','의류_중').to_csv('의류_중_구매지수(1-7).csv',encoding='cp949',index=False)
get_category('test','sum','의류_하').to_csv('의류_하_구매지수(1-7).csv',encoding='cp949',index=False)


In [21]:
get_category('train','sum','신선식품_상').to_csv('신선식품_상_구매지수(1-6).csv',encoding='cp949',index=False)
get_category('train','sum','신선식품_중').to_csv('신선식품_중_구매지수(1-6).csv',encoding='cp949',index=False)
get_category('train','sum','신선식품_하').to_csv('신선식품_하_구매지수(1-6).csv',encoding='cp949',index=False)

get_category('test','sum','신선식품_상').to_csv('신선식품_상_구매지수(1-7).csv',encoding='cp949',index=False)
get_category('test','sum','신선식품_중').to_csv('신선식품_중_구매지수(1-7).csv',encoding='cp949',index=False)
get_category('test','sum','신선식품_하').to_csv('신선식품_하_구매지수(1-7).csv',encoding='cp949',index=False)

In [17]:
get_category('train','count','신선식품_상').to_csv('신선식품_상_방문지수(1-6).csv',encoding='cp949',index=False)
get_category('train','count','신선식품_중').to_csv('신선식품_중_방문지수(1-6).csv',encoding='cp949',index=False)
get_category('train','count','신선식품_하').to_csv('신선식품_하_방문지수(1-6).csv',encoding='cp949',index=False)

get_category('test','count','신선식품_상').to_csv('신선식품_상_방문지수(1-7).csv',encoding='cp949',index=False)
get_category('test','count','신선식품_중').to_csv('신선식품_중_방문지수(1-7).csv',encoding='cp949',index=False)
get_category('test','count','신선식품_하').to_csv('신선식품_하_방문지수(1-7).csv',encoding='cp949',index=False)